In [ ]:
import whois
import time
import util.util as util
import socket
import whois
from tld import get_tld


In [ ]:
# TODO: set the path to your result folder
# set the first argument to the base path of the result folder, second argument is th folder name, third argument a mapping file - that tells for apps in multiple sub datasets which app to include
verified = util.get_verified_dataset("/", "/2023_04_06/", "/verified_dataset/result.json")

In [ ]:
def getAllDataWithoutRequests(app):
    result = set()
    result.update(util.getUniqueDomainsFromJson(app.amqp))
    result.update(util.getUniqueDomainsFromJson(app.coap))
    result.update(util.getUniqueDomainsFromJson(app.endpoints))
    result.update(util.getUniqueDomainsFromJson(app.mqtt))
    result.update(util.getUniqueDomainsFromJson(app.xmpp))
    return result


In [ ]:
apps = []
for a in verified:
    apps.append(util.domainsWithoutLocal(getAllDataWithoutRequests(a)))

In [ ]:
getAllDataWithoutRequests(appAnalysis[3])

In [ ]:
def notResolving(hostname):
    try:
        return socket.gethostbyname(hostname) == None
    except UnicodeError as e:
        return True
    except socket.gaierror as e:
        return True

In [ ]:
allDomains = set()
for a in apps:
    for d in a:
        allDomains.add(d)

In [ ]:
notResolvingDomains = set()
for d in allDomains:
    if notResolving(d):
        notResolvingDomains.add(d)

In [ ]:
filteredDomains = set()
likelyInvalid = ["GROUP", "zip", None, "name", "android", "mp"]
for d in notResolvingDomains:
    if any(ele.isupper() for ele in d) or d[0] == '.' or d[0].isdigit():
        continue
    if get_tld(d, fix_protocol=True, fail_silently=True) not in likelyInvalid:
        filteredDomains.add(d)


In [ ]:
def isNotAvailable(domain):
    try:
        w = whois.whois(domain)
        if w.domain_name == None:
            return True
            #print(w)
    except whois.parser.PywhoisError:
        #print("Exception:")
        return True
        #time.sleep(5)
    return False


In [ ]:
result = set()
for d in filteredDomains:
    if isNotAvailable(d):
        result.add(d)



In [ ]:
result

In [ ]:
len(result)